## Catching a Falling Initial

In this notebook, we will learn how to catch an object that is falling with a Kuka iiwa-7. This task is difficult because of the timing components. You will be responsible for calculating the rate at which an object is falling using several Intel Realsense cameras and then calculating the expected position of the object at a certain point in time, where you will catch the object.


**Learning Objectives:**
1. Implement a system that uses a known location of a falling item to catch a falling initial
2. Implement control (probably force control) to catch a falling object without it hitting the ground.

## Setup and imports
Let us first get our imports out of the way:

In [ ]:
import os
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import trimesh
from pydrake.all import (
    AddFrameTriadIllustration,
    BasicVector,
    Concatenate,
    Context,
    Diagram,
    DiagramBuilder,
    Integrator,
    JacobianWrtVariable,
    LeafSystem,
    MultibodyPlant,
    PiecewisePolynomial,
    PiecewisePose,
    RigidTransform,
    RobotDiagram,
    RollPitchYaw,
    RotationMatrix,
    Simulator,
    StartMeshcat,
    Trajectory,
    TrajectorySource,
)

from manipulation import running_as_notebook
from manipulation.exercises.grader import Grader
from manipulation.icp import IterativeClosestPoint
from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import (
    AddPointClouds,
    LoadScenario,
    MakeHardwareStation,
    RobotDiagram,
)
from manipulation.utils import RenderDiagram

### Meshcat Visualization

As always, let's start Meshcat for our 3D visualization!

In [39]:
# Start meshcat for visualization
meshcat = StartMeshcat()
print("Click the link above to open Meshcat in your browser!")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Click the link above to open Meshcat in your browser!


In [342]:
# A note: as an additional argument add `include_normals=True` to the `create_sdf_asset_from_letter` function.
# We also recommend you use a slightly smaller `letter_height_meters=0.15` and a rather high friction coefficient
# (we use mu_static = 1.17, mu_dynamic = 1.0 - roughly equal to the friction of rubber on granite) to make the
# letters easier to pick up. We'll study more complicated grasping strategies in future notebooks, but this is not
# the priority of this chapter so we'll cheat.

from pathlib import Path
from manipulation.letter_generation import create_sdf_asset_from_letter

output_dir = Path("assets")
output_dir.mkdir(parents=True, exist_ok=True)

your_initial = "I"

create_sdf_asset_from_letter(
    your_initial,             # just the letter as the first positional arg
    output_dir=output_dir,    # keep this as a keyword
    include_normals=True,     # might be include_normal in your version
    letter_height_meters=0.25,
    mu_static=1.17,
    mu_dynamic=1.0,
)

PosixPath('assets/I.sdf')

In [343]:
# Creating a Table SDF that the letter will fall onto (potentially if I have time, I'll setit up to fall somewhere randomly in this space)
size_x = 2
size_y = 2
thickness = 0.05
mass = 10.0
mu_static = 1.0
mu_dynamic = 0.8
dissipation = 1.0

Ixx = (1.0/12.0) * mass * (size_y**2 + thickness**2)
Iyy = (1.0/12.0) * mass * (size_x**2 + thickness**2)
Izz = (1.0/12.0) * mass * (size_x**2 + size_y**2)

table_sdf = f"""<?xml version="1.0"?>
<sdf version="1.7" xmlns:drake="http://drake.mit.edu">
  <model name="table">
    <link name="table_top">
      <inertial>
        <mass>{mass}</mass>
        <inertia>
          <ixx>{Ixx}</ixx><iyy>{Iyy}</iyy><izz>{Izz}</izz>
          <ixy>0</ixy><ixz>0</ixz><iyz>0</iyz>
        </inertia>
      </inertial>

      <visual name="visual">
        <geometry><box><size>{size_x} {size_y} {thickness}</size></box></geometry>
        <material>
          <ambient>0.6 0.6 0.6 1</ambient>
          <diffuse>0.7 0.7 0.7 1</diffuse>
        </material>
      </visual>

      <collision name="collision">
        <geometry><box><size>{size_x} {size_y} {thickness}</size></box></geometry>
        <drake:proximity_properties>
          <drake:hunt_crossley_dissipation>{dissipation}</drake:hunt_crossley_dissipation>
          <drake:mu_static>{mu_static}</drake:mu_static>
          <drake:mu_dynamic>{mu_dynamic}</drake:mu_dynamic>
        </drake:proximity_properties>
      </collision>
    </link>
  </model>
</sdf>

"""

assets_dir = Path("assets")
assets_dir.mkdir(parents=True, exist_ok=True)
(assets_dir / "table.sdf").write_text(table_sdf)
print("Wrote", (assets_dir / "table.sdf").resolve())

Wrote /workspaces/Projects/robotic_manipulation/00-Pset-9/assets/table.sdf


In [352]:
# Add the directives for the bimanual IIWA arms, table, and initials

def generate_bimanual_IIWA14_with_assets_directives_file() -> (
    tuple[Diagram, RobotDiagram]
):
    table_sdf = f"{Path.cwd()}/assets/table.sdf"
    letter_sdf = f"{Path.cwd()}/assets/{your_initial}.sdf"

    directives_yaml = f"""directives:
- add_model:
    name: table
    file: file://{table_sdf}
- add_weld:
    parent: world
    child: table::table_top
    X_PC:
        translation: [0.0, 0.0, -.025]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [0.0]
        iiwa_joint_2: [0.5]
        iiwa_joint_3: [0.0]
        iiwa_joint_4: [-1.8]
        iiwa_joint_5: [0.0]
        iiwa_joint_6: [1.2]
        iiwa_joint_7: [0.0]
- add_weld:
    parent: table::table_top
    child: iiwa::iiwa_link_0
    X_PC:
        translation: [.5, 0, .025]
        rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
        translation: [0, 0, 0.09]
        rotation: !Rpy {{ deg: [90, 0, 90]}}



- add_model:
    name: {your_initial}_letter
    file: file://{letter_sdf}


"""
    os.makedirs("directives", exist_ok=True)

    with open(
        "directives/bimanual_IIWA14_with_table_and_initials_and_assets.dmd.yaml", "w"
    ) as f:
        f.write(directives_yaml)




        # iiwa_joint_1: [-1.57]
        # iiwa_joint_2: [0.1]
        # iiwa_joint_3: [0]
        # iiwa_joint_4: [-1.2]
        # iiwa_joint_5: [0]
        # iiwa_joint_6: [1.6]
        # iiwa_joint_7: [0]

        # iiwa_joint_1: [0]
        # iiwa_joint_2: [0]
        # iiwa_joint_3: [0]
        # iiwa_joint_4: [0]
        # iiwa_joint_5: [0]
        # iiwa_joint_6: [0]
        # iiwa_joint_7: [0]

        # iiwa_joint_1: [0.0]
        # iiwa_joint_2: [0.5]
        # iiwa_joint_3: [0.0]
        # iiwa_joint_4: [-1.8]
        # iiwa_joint_5: [0.0]
        # iiwa_joint_6: [1.2]
        # iiwa_joint_7: [0.0]


# - add_model:
#     name: cheez_it
#     file: package://manipulation/hydro/003_cracker_box.sdf
generate_bimanual_IIWA14_with_assets_directives_file()

In [353]:
def create_bimanual_IIWA14_with_assets_scenario() -> None:
    # Build absolute file:// URIs for the two directives files
    dmd1 = (Path.cwd() / "directives" / "bimanual_IIWA14_with_table_and_initials_and_assets.dmd.yaml").resolve().as_posix()
    # dmd2 = (Path.cwd() / "directives" / "camera_directives.dmd.yaml").resolve().as_posix()

    # Part 1: directives (use file:// URIs)
    scenario_yaml = f"""# Station scenario
directives:
  - add_directives:
      file: file://{dmd1}
"""
#   - add_directives:
#       file: file://{dmd2}



    # Part 2: cameras (DICT form; fields your schema accepts)
#     scenario_yaml += """
# cameras:
#   camera0:
#     name: camera0
#     depth: true
#     X_PB:
#       base_frame: camera0::base
#   camera1:
#     name: camera1
#     depth: true
#     X_PB:
#       base_frame: camera1::base
#   camera2:
#     name: camera2
#     depth: true
#     X_PB:
#       base_frame: camera2::base
# """

    # Part 3: drivers
    scenario_yaml += """
model_drivers:
  iiwa: !IiwaDriver
    control_mode: position_only
    hand_model_name: wsg
  wsg: !SchunkWsgDriver {}
"""

    os.makedirs("scenarios", exist_ok=True)

    with open(
        "scenarios/bimanual_IIWA14_with_table_and_initials_and_assets.scenario.yaml",
        "w",
    ) as f:
        f.write(scenario_yaml)


create_bimanual_IIWA14_with_assets_scenario()

In [354]:
def create_bimanual_IIWA14_with_table_and_initials_and_assets() -> (
    tuple[DiagramBuilder, RobotDiagram]
):
    # TODO: Load the scenario created above into a Scenario object
    scenario = LoadScenario(
        filename="scenarios/bimanual_IIWA14_with_table_and_initials_and_assets.scenario.yaml"
    )

    # TODO: Create HardwareStation with the scenario and meshcat
    station = MakeHardwareStation(scenario, meshcat=meshcat)

    # TODO: Make a DiagramBuilder, add the station, and build the diagram
    builder = DiagramBuilder()
    builder.AddSystem(station)

    # TODO: Add the point clouds to the diagram with AddPointClouds
    ports = AddPointClouds(scenario=scenario, station=station, builder=builder)

    # TODO: export the point cloud outputs to the builder
    if ports:
        for cam_name, pc_sys in ports.items():
            # pc_sys is a pydrake.perception.DepthImageToPointCloud System
            # Export its point cloud output
            try:
                # Newer Drake often names it "cloud"
                builder.ExportOutput(pc_sys.get_output_port("cloud"),
                                    f"{cam_name}.point_cloud")
            except Exception:
                # Fallback: use the first output port
                builder.ExportOutput(pc_sys.get_output_port(0),
                                    f"{cam_name}.point_cloud")

    # TODO: Return the builder AND the station (notice that here we will need both)
    return builder, station


In [355]:
# builder, station = (
#     create_bimanual_IIWA14_with_table_and_initials_and_assets()
# )

# # in order to debug, we will build the diagram once here.
# from pydrake.all import ConstantVectorSource
# force_source = builder.AddSystem(ConstantVectorSource([100.0]))
# builder.Connect(force_source.get_output_port(), station.GetInputPort("wsg.force_limit"))
# diagram = builder.Build()



# # visualize the diagram
# RenderDiagram(diagram, max_depth=1)

# # publish the diagram with some default context
# context = diagram.CreateDefaultContext()



# diagram.ForcedPublish(context)


In [ ]:
# TODO: modify the functions below to work with your letters. This will require quite a bit of trial and error.


def design_grasp_pose(X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    R_OG = (
        RollPitchYaw(0, 0, np.pi).ToRotationMatrix()
        @ RollPitchYaw(-np.pi / 2, 0, 0).ToRotationMatrix()
    )
    p_OG = [0.07, 0.08, 0.12]
    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO.multiply(X_OG)
    return X_OG, X_WG

# def design_grasp_pose_for_catching(X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
#     """
#     Design gripper pose to catch falling object.
#     Gripper approaches from below with fingers vertical.
#     """
#     # Gripper frame relative to object frame
#     # Keep it simple: gripper directly below object center
#     R_OG = RotationMatrix.MakeXRotation(np.pi)  # Fingers pointing up
#     p_OG = [0.0, 0.0, -0.08]  # 8cm below object center (adjust for your letter size)

#     X_OG = RigidTransform(R_OG, p_OG)
#     X_WG = X_WO @ X_OG
#     return X_OG, X_WG

def design_grasp_pose_for_catching(X_WO: RigidTransform) -> tuple[RigidTransform, RigidTransform]:
    """
    Design gripper pose to catch falling object.

    Key insight: The WSG gripper on the IIWA has fingers that:
    - Open/close along the gripper's +/- Y axis
    - The gripper's Z axis points "forward" (approach direction)
    - The gripper's X axis is the finger separation axis

    For catching a falling letter:
    - We want the gripper below the object
    - Fingers should be horizontal (perpendicular to gravity)
    - Approach from below means gripper Z should point up (+Z in world)
    """

    # Step 1: Get the letter's orientation
    # Assuming letter is oriented with its flat face in XY plane as it falls
    R_WO = X_WO.rotation()
    p_WO = X_WO.translation()

    # Step 2: Design gripper orientation
    # We want:
    # - Gripper Z axis pointing UP (towards the falling letter)
    # - Gripper Y axis horizontal (for finger opening)
    # - This means gripper X axis is also horizontal

    # Option A: If letter maintains upright orientation as it falls
    # Gripper approaches from directly below, fingers open horizontally
    R_OG = RotationMatrix.MakeXRotation(np.pi)  # Flip upside down

    # Step 3: Position gripper below the object center
    # Adjust this offset based on your letter size and gripper geometry
    # Negative Z means "below" in the object frame
    letter_half_height = 0.075  # Half of 0.15m letter height
    gripper_clearance = 0.02    # Small gap for safety
    p_OG = np.array([0.0, 0.0, -(letter_half_height + gripper_clearance)])

    X_OG = RigidTransform(R_OG, p_OG)
    X_WG = X_WO @ X_OG
    return X_OG, X_WG

def design_pregrasp_pose(X_WG: RigidTransform) -> RigidTransform:
    """
    Create a pre-grasp pose that's offset from the grasp pose.
    For catching, we want to approach from below and slightly to the side.
    """
    # Move back along the gripper's -Z axis (away from approach direction)
    # and down a bit more in world Z
    X_GGpre = RigidTransform(
        RotationMatrix(),  # Same orientation
        np.array([0.0, 0.0, -0.15])  # 15cm back along gripper Z axis
    )

    X_WGpre = X_WG @ X_GGpre

    # Additional offset: lower in world frame for safer approach
    # IMPORTANT: Make a copy since translation() returns read-only array
    p_WGpre = X_WGpre.translation().copy()
    p_WGpre[2] -= 0.05  # 5cm lower in world Z

    return RigidTransform(X_WGpre.rotation(), p_WGpre)

def design_pregoal_pose(
    X_WG: RigidTransform,
) -> tuple[RigidTransform, RigidTransform, RigidTransform]:
    X_GGApproach = RigidTransform([0.0, 0.0, -0.2])
    X_WGApproach = X_WG @ X_GGApproach
    return X_WGApproach



def verify_grasp_geometry(X_WO_catch: RigidTransform, X_WG_catch: RigidTransform):
    """
    Debug helper: Check if gripper and object poses make sense.
    """
    p_WO = X_WO_catch.translation()
    p_WG = X_WG_catch.translation()

    print("\n=== GRASP GEOMETRY CHECK ===")
    print(f"Object position: [{p_WO[0]:.3f}, {p_WO[1]:.3f}, {p_WO[2]:.3f}]")
    print(f"Gripper position: [{p_WG[0]:.3f}, {p_WG[1]:.3f}, {p_WG[2]:.3f}]")
    print(f"Offset: [{p_WG[0]-p_WO[0]:.3f}, {p_WG[1]-p_WO[1]:.3f}, {p_WG[2]-p_WO[2]:.3f}]")

    # Check if gripper is below object (for catching from below)
    if p_WG[2] < p_WO[2]:
        print("✓ Gripper is below object (correct for catching)")
    else:
        print("✗ WARNING: Gripper is ABOVE object!")

    # Check gripper Z axis direction (should point up for catching)
    z_G = X_WG_catch.rotation().matrix()[:, 2]  # Third column = Z axis
    print(f"Gripper Z axis in world: [{z_G[0]:.3f}, {z_G[1]:.3f}, {z_G[2]:.3f}]")
    if z_G[2] > 0.5:  # Pointing mostly upward
        print("✓ Gripper Z axis points up (correct for catching)")
    else:
        print("✗ WARNING: Gripper Z axis not pointing up!")
    print("===========================\n")



def make_bot_trajectory(
    X_Gs: list[RigidTransform], finger_values: np.ndarray, sample_times: list[float]
) -> tuple[Trajectory, PiecewisePolynomial]:
    robot_position_trajectory = PiecewisePose.MakeLinear(sample_times, X_Gs)
    robot_velocity_trajectory = robot_position_trajectory.MakeDerivative()
    traj_wsg_command = PiecewisePolynomial.FirstOrderHold(sample_times, finger_values)
    return robot_velocity_trajectory, traj_wsg_command

In [357]:
# TODO: copy over the pseudoinverse controller from the previous pset below.
# OPTIONAL: if you want to you can try designing a more complicated controller for the pick and place.


class PseudoInverseController(LeafSystem):
    def __init__(self, plant: MultibodyPlant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_WG", 6)
        self.q_port = self.DeclareVectorInputPort("iiwa.position", 7)
        self.DeclareVectorOutputPort("iiwa.velocity", 7, self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context: Context, output: BasicVector):
        """
        fill in our code below.
        """
        # evaluate the V_G_port and q_port on the current context to get those values.
        V_G = V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)

        # update the positions of the internal _plant_context according to `q`.
        # HINT: you can write to a plant context by calling `self._plant.SetPositions`

        self._plant.SetPositions(self._plant_context, self._iiwa, q)

        # Compute the gripper jacobian
        # HINT: the jacobian is 6 x N, with N being the number of DOFs.
        # We only want the 6 x 7 submatrix corresponding to the IIWA
        J_full = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context,
            JacobianWrtVariable.kV,
            self._G, np.zeros(3),   # p_BoQ_B = [0,0,0] at frame G origin
            self._W,                # measured in world (Ao = W)
            self._W                 # expressed in world (E = W)
        )
        J = J_full[:, self.iiwa_start : self.iiwa_end + 1]

        # compute `v` by mapping the gripper velocity (from the V_G_port) to the joint space
        lam2 = 1e-6
        JJt = J @ J.T
        J_pinv = J.T @ np.linalg.inv(JJt + lam2 * np.eye(6))
        v = J_pinv @ V_G  # 7x1 joint velocity command

        output.SetFromVector(v)

In [358]:
builder, station = (
    create_bimanual_IIWA14_with_table_and_initials_and_assets()
)
plant = station.GetSubsystemByName("plant")

station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)

# get initial poses of gripper
X_WGinitial = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body"))


# Get the letter body
letter_model = plant.GetModelInstanceByName(f"{your_initial}_letter")
letter_body = plant.GetBodyByName(f"{your_initial}_body_link", letter_model)

# IMPORTANT: You need to know where the letter STARTS
# This should match wherever you set it in the simulation setup
p0_letter = np.array([0.0, 0.0, 10])  # Initial position (adjust to match your setup)
v0_letter = np.array([0.0, 0.0, 0.0])  # Starts from rest

# Physics prediction
catch_height = 0.25  # Where you want to catch it (meters above table)
g = -9.81

# Solve for time: catch_height = p0_z + v0_z*t + 0.5*g*t^2
# Rearranged: 0.5*g*t^2 + v0_z*t + (p0_z - catch_height) = 0
def solve_catch_time(z0, vz0, g, z_target):
    """Solve quadratic equation for time to reach target height"""
    a = 0.5 * g
    b = vz0
    c = z0 - z_target

    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        print("WARNING: Object won't reach target height!")
        return None

    t1 = (-b + np.sqrt(discriminant)) / (2*a)
    t2 = (-b - np.sqrt(discriminant)) / (2*a)

    # Take the positive time
    valid_times = [t for t in [t1, t2] if t > 0]
    return min(valid_times) if valid_times else None

t_catch = solve_catch_time(p0_letter[2], v0_letter[2], g, catch_height)

if t_catch is None:
    print("ERROR: Can't calculate catch time!")
    t_catch = 1.0  # Fallback

print(f"Letter will reach {catch_height}m in {t_catch:.2f} seconds")

# Predict where letter will be at catch time
p_catch = p0_letter + v0_letter * t_catch + 0.5 * np.array([0, 0, g]) * t_catch**2

print(f"Predicted catch location: x={p_catch[0]:.3f}, y={p_catch[1]:.3f}, z={p_catch[2]:.3f}")

# Create the catch pose (object pose at catch time)
# Assuming letter stays upright as it falls
R_catch = RotationMatrix.MakeXRotation(np.pi/2)  # Adjust based on your letter orientation
X_WO_catch = RigidTransform(R_catch, p_catch)

# ============================================================
# NOW USE THIS PREDICTED POSE FOR YOUR KEYFRAMES
# ============================================================

# Design grasp pose based on predicted catch location
X_OG, X_WGpick = design_grasp_pose_for_catching(X_WO_catch)
X_WGprepick = design_pregrasp_pose(X_WGpick)

# constants for finger distances when the gripper is opened or closed
opened = 0.107
closed = 0.0

# Build trajectory keyframes
# Timing is critical here - gripper needs to arrive BEFORE the letter
# Drake requires at least 0.1s between time steps

# Calculate safe times with minimum spacing
min_spacing = 0.15  # Minimum time between waypoints (Drake needs >= 0.1s)

if t_catch < 2.0:
    print(f"WARNING: Catch time {t_catch:.2f}s is very short!")
    # Scale everything faster
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, max(0.5, t_catch * 0.3)),
        ("X_WGpick", X_WGpick, opened, max(0.8, t_catch * 0.7)),
        ("X_WGpick", X_WGpick, closed, t_catch + 0.2),
        ("X_WGpick", X_WGpick, closed, t_catch + 1.5),  # Hold
    ]
else:
    # Normal timing
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, t_catch - 1.5),
        ("X_WGpick", X_WGpick, opened, t_catch - 0.3),  # Arrive early
        ("X_WGpick", X_WGpick, closed, t_catch + 0.2),  # Close after catch
        ("X_WGpick", X_WGpick, closed, t_catch + 1.5),  # Hold
    ]

# Validate times are strictly increasing with enough spacing
sample_times = [keyframe[3] for keyframe in keyframes]
for i in range(1, len(sample_times)):
    if sample_times[i] - sample_times[i-1] < min_spacing:
        print(f"ERROR: Times too close: {sample_times[i-1]:.2f} -> {sample_times[i]:.2f}")
        # Fix by spreading them out
        sample_times[i] = sample_times[i-1] + min_spacing

print(f"Trajectory times: {sample_times}")
print(f"Letter catches at t={t_catch:.2f}s")

# Extract poses, finger states, and TIMES from keyframes
gripper_poses = [keyframe[1] for keyframe in keyframes]
finger_states = np.asarray([keyframe[2] for keyframe in keyframes]).reshape(1, -1)
# sample_times already validated above

traj_V_G, traj_wsg_command = make_bot_trajectory(gripper_poses, finger_states, sample_times)

# ============================================================
# REST OF YOUR CODE STAYS THE SAME
# ============================================================

# V_G_source defines a trajectory over gripper velocities. Add it to the system.
V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
# Add the DiffIK controller we just defined to the system
controller = builder.AddSystem(PseudoInverseController(plant))
# The HardwareStation expects robot commands in terms of joint angles.
# We define the `integrator` system to map from joint_velocities to joint_angles.
integrator = builder.AddSystem(Integrator(7))
# wsg_source defines a trajectory of finger positions. Add it to the system.
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# Connect everything
builder.Connect(V_G_source.get_output_port(), controller.GetInputPort("V_WG"))
builder.Connect(controller.GetOutputPort("iiwa.velocity"), integrator.get_input_port())
builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(station.GetOutputPort("iiwa.position_measured"), controller.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(), station.GetInputPort("wsg.position"))

# visualize axes (useful for debugging)
scenegraph = station.GetSubsystemByName("scene_graph")
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName(f"{your_initial}_body_link"),
    length=0.1,
)
AddFrameTriadIllustration(
    scene_graph=scenegraph, body=plant.GetBodyByName("body"), length=0.1
)

diagram = builder.Build()


Letter will reach 0.25m in 1.41 seconds
Predicted catch location: x=0.000, y=0.000, z=0.250
Trajectory times: [0.0, 0.5, np.float64(0.9869174819649061), np.float64(1.609882117092723), np.float64(2.909882117092723)]
Letter catches at t=1.41s


In [359]:
# Replace your current simulation setup code (the last cell) with this:

# Define the simulator
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyContextFromRoot(context)

# Get the plant context from the simulator's context
plant_context = plant.GetMyMutableContextFromRoot(context)

# NOW set the letter pose in the simulation context
letter_model = plant.GetModelInstanceByName(f"{your_initial}_letter")
letter_body = plant.GetBodyByName(f"{your_initial}_body_link", letter_model)

# Set your desired initial pose
R_WL = RollPitchYaw(np.pi/2, 0, 0)  # 90° roll
p_WL = p0_letter  # x, y, z in world (adjust height as needed)

plant.SetFreeBodyPose(
    plant_context,
    letter_body,
    RigidTransform(R_WL, p_WL),
)

# Set integrator initial value
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    plant.GetPositions(
        plant_context,
        plant.GetModelInstanceByName("iiwa"),
    ),
)

# Force publish to update visualization
diagram.ForcedPublish(context)
print(f"Simulation will run for {traj_V_G.end_time()} seconds")

# Run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(traj_V_G.end_time() if running_as_notebook else 0.01)
meshcat.StopRecording()
meshcat.PublishRecording()

Simulation will run for 2.909882117092723 seconds


In [ ]:
# Code block 1

builder, station = (
    create_bimanual_IIWA14_with_table_and_initials_and_assets()
)
plant = station.GetSubsystemByName("plant")

station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)

# get initial poses of gripper
X_WGinitial = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body"))


# Get the letter body
letter_model = plant.GetModelInstanceByName(f"{your_initial}_letter")
letter_body = plant.GetBodyByName(f"{your_initial}_body_link", letter_model)

# IMPORTANT: You need to know where the letter STARTS
# This should match wherever you set it in the simulation setup
p0_letter = np.array([0.0, 0.0, 25])  # Initial position (adjust to match your setup)
v0_letter = np.array([0.0, 0.0, 0.0])  # Starts from rest

# Physics prediction
catch_height = 1.2  # Where you want to catch it (meters above table)
g = -9.81

# Solve for time: catch_height = p0_z + v0_z*t + 0.5*g*t^2
# Rearranged: 0.5*g*t^2 + v0_z*t + (p0_z - catch_height) = 0
def solve_catch_time(z0, vz0, g, z_target):
    """Solve quadratic equation for time to reach target height"""
    a = 0.5 * g
    b = vz0
    c = z0 - z_target

    discriminant = b**2 - 4*a*c
    if discriminant < 0:
        print("WARNING: Object won't reach target height!")
        return None

    t1 = (-b + np.sqrt(discriminant)) / (2*a)
    t2 = (-b - np.sqrt(discriminant)) / (2*a)

    # Take the positive time
    valid_times = [t for t in [t1, t2] if t > 0]
    return min(valid_times) if valid_times else None

t_catch = solve_catch_time(p0_letter[2], v0_letter[2], g, catch_height)

if t_catch is None:
    print("ERROR: Can't calculate catch time!")
    t_catch = 1.0  # Fallback

print(f"Letter will reach {catch_height}m in {t_catch:.2f} seconds")

# Predict where letter will be at catch time
p_catch = p0_letter + v0_letter * t_catch + 0.5 * np.array([0, 0, g]) * t_catch**2

print(f"Predicted catch location: x={p_catch[0]:.3f}, y={p_catch[1]:.3f}, z={p_catch[2]:.3f}")

R_catch = RotationMatrix.MakeXRotation(np.pi/2)  # Or whatever your letter orientation is
X_WO_catch = RigidTransform(R_catch, p_catch)

# Design gripper poses - now much simpler!
X_OG, X_WGpick = design_grasp_pose_for_catching(X_WO_catch)
X_WGprepick = design_pregrasp_pose(X_WGpick)

# Debug it
verify_grasp_geometry(X_WO_catch, X_WGpick)


# constants for finger distances when the gripper is opened or closed
opened = 0.107
closed = 0.0

# Build trajectory keyframes
# Timing is critical here - gripper needs to arrive BEFORE the letter
# Drake requires at least 0.1s between time steps

# Calculate safe times with minimum spacing
min_spacing = 0.15  # Minimum time between waypoints (Drake needs >= 0.1s)

if t_catch < 2.0:
    print(f"WARNING: Catch time {t_catch:.2f}s is very short!")
    # Scale everything faster
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, max(0.5, t_catch * 0.3)),
        ("X_WGpick", X_WGpick, opened, max(0.8, t_catch * 0.7)),
        ("X_WGpick", X_WGpick, closed, t_catch + 0.2),
        ("X_WGpick", X_WGpick, closed, t_catch + 1.5),  # Hold
    ]
else:
    # Normal timing
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, t_catch - 1.5),
        ("X_WGpick", X_WGpick, opened, t_catch - 0.3),  # Arrive early
        ("X_WGpick", X_WGpick, closed, t_catch + 0.2),  # Close after catch
        ("X_WGpick", X_WGpick, closed, t_catch + 1.5),  # Hold
    ]

# Validate times are strictly increasing with enough spacing
sample_times = [keyframe[3] for keyframe in keyframes]
for i in range(1, len(sample_times)):
    if sample_times[i] - sample_times[i-1] < min_spacing:
        print(f"ERROR: Times too close: {sample_times[i-1]:.2f} -> {sample_times[i]:.2f}")
        # Fix by spreading them out
        sample_times[i] = sample_times[i-1] + min_spacing

print(f"Trajectory times: {sample_times}")
print(f"Letter catches at t={t_catch:.2f}s")

# Extract poses, finger states, and TIMES from keyframes
gripper_poses = [keyframe[1] for keyframe in keyframes]
finger_states = np.asarray([keyframe[2] for keyframe in keyframes]).reshape(1, -1)
# sample_times already validated above

traj_V_G, traj_wsg_command = make_bot_trajectory(gripper_poses, finger_states, sample_times)

# ============================================================
# REST OF YOUR CODE STAYS THE SAME
# ============================================================

# V_G_source defines a trajectory over gripper velocities. Add it to the system.
V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
# Add the DiffIK controller we just defined to the system
controller = builder.AddSystem(PseudoInverseController(plant))
# The HardwareStation expects robot commands in terms of joint angles.
# We define the `integrator` system to map from joint_velocities to joint_angles.
integrator = builder.AddSystem(Integrator(7))
# wsg_source defines a trajectory of finger positions. Add it to the system.
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# Connect everything
builder.Connect(V_G_source.get_output_port(), controller.GetInputPort("V_WG"))
builder.Connect(controller.GetOutputPort("iiwa.velocity"), integrator.get_input_port())
builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(station.GetOutputPort("iiwa.position_measured"), controller.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(), station.GetInputPort("wsg.position"))

# visualize axes (useful for debugging)
scenegraph = station.GetSubsystemByName("scene_graph")
AddFrameTriadIllustration(
    scene_graph=scenegraph,
    body=plant.GetBodyByName(f"{your_initial}_body_link"),
    length=0.1,
)
AddFrameTriadIllustration(
    scene_graph=scenegraph, body=plant.GetBodyByName("body"), length=0.1
)

diagram = builder.Build()


In [ ]:
#  Code block 2



# Define the simulator
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyContextFromRoot(context)

# Get the plant context from the simulator's context
plant_context = plant.GetMyMutableContextFromRoot(context)

# NOW set the letter pose in the simulation context
letter_model = plant.GetModelInstanceByName(f"{your_initial}_letter")
letter_body = plant.GetBodyByName(f"{your_initial}_body_link", letter_model)



# Set your desired initial pose
R_WL = RollPitchYaw(np.pi/2, 0, 0)  # 90° roll
p_WL = p0_letter  # x, y, z in world (adjust height as needed)

plant.SetFreeBodyPose(
    plant_context,
    letter_body,
    RigidTransform(R_WL, p_WL),
)






# Set integrator initial value
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    plant.GetPositions(
        plant_context,
        plant.GetModelInstanceByName("iiwa"),
    ),
)

# Force publish to update visualization
diagram.ForcedPublish(context)
print(f"Simulation will run for {traj_V_G.end_time()} seconds")

# Run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
simulator.AdvanceTo(traj_V_G.end_time() if running_as_notebook else 0.01)
meshcat.StopRecording()
meshcat.PublishRecording()

In [ ]:
builder, station = (
    create_bimanual_IIWA14_with_table_and_initials_and_assets()
)
plant = station.GetSubsystemByName("plant")

station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)



# get initial poses of gripper
X_WGinitial = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body"))
print(X_WGinitial)

letter_model, letter_body = get_model_body(plant)

# Set the letter position to YOUR desired value (overwrites SDF/URDF)
p0_letter = np.array([0.0, 0.0, 21.5])  # YOUR desired initial position
R_WL = RollPitchYaw(np.pi/2, np.pi, np.pi/2)  # Your desired orientation
plant.SetFreeBodyPose(
    plant_context,
    letter_body,
    RigidTransform(R_WL, p0_letter),
)
v0_letter = np.array([0.0, 0.0, 0.0])  # Starts from rest
catch_height = 1.5


# Physics prediction


t_catch = solve_catch_time(p0_letter[2], v0_letter[2], catch_height)

if t_catch is None:
    print("ERROR: Can't calculate catch time!")
    t_catch = 1.0  # Fallback

print(f"Letter will reach {catch_height}m in {t_catch:.2f} seconds")
g = -9.81
# Predict where letter will be at catch time
p_catch = p0_letter + v0_letter * t_catch + 0.5 * np.array([0, 0, g]) * t_catch**2

print(f"Predicted catch location: x={p_catch[0]:.3f}, y={p_catch[1]:.3f}, z={p_catch[2]:.3f}")

R_catch = RotationMatrix.MakeZRotation(np.pi/2)  # Or whatever your letter orientation is
X_WO_catch = RigidTransform(R_catch, p_catch)



# EDIT HERE TO DO

# Design gripper poses - now much simpler!
X_OG, X_WGpick = design_grasp_pose_for_catching(X_WO_catch)
X_WGprepick = design_pregrasp_pose(X_WGpick)


# Debug it
verify_grasp_geometry(X_WO_catch, X_WGpick)


# constants for finger distances when the gripper is opened or closed
opened = 0.107
closed = 0.0


# Calculate safe times with minimum spacing
min_spacing = 0.15  # Minimum time between waypoints (Drake needs >= 0.1s)

print("X_WGpick: ")
print( X_WGpick)



print("DONE")

if t_catch < 2.0:
    print(f"WARNING: Catch time {t_catch:.2f}s is very short!")
    # Scale everything faster
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, max(0.5, t_catch * 0.3)),
        ("X_WGpick", X_WGpick, opened, max(0.8, t_catch * 0.7)),
        ("X_WGpick", X_WGpick, closed, t_catch + 0.2),
        ("X_WGpick", X_WGpick, closed, t_catch + 1.5),  # Hold
    ]
else:
    # Normal timing
    keyframes = [
        ("X_WGinitial", X_WGinitial, opened, 0.0),
        ("X_WGprepick", X_WGprepick, opened, t_catch - 1.5),
        ("X_WGpick", X_WGpick, opened, t_catch - 0.3),  # Arrive early
        ("X_WGpick", X_WGpick, closed, t_catch + 0.2),  # Close after catch
        ("X_WGpick", X_WGpick, closed, t_catch + 1.5),  # Hold
    ]

# Validate times are strictly increasing with enough spacing
sample_times = [keyframe[3] for keyframe in keyframes]
for i in range(1, len(sample_times)):
    if sample_times[i] - sample_times[i-1] < min_spacing:
        print(f"ERROR: Times too close: {sample_times[i-1]:.2f} -> {sample_times[i]:.2f}")
        # Fix by spreading them out
        sample_times[i] = sample_times[i-1] + min_spacing

print(f"Trajectory times: {sample_times}")
print(f"Letter catches at t={t_catch:.2f}s")

# Extract poses, finger states, and TIMES from keyframes
gripper_poses = [keyframe[1] for keyframe in keyframes]
finger_states = np.asarray([keyframe[2] for keyframe in keyframes]).reshape(1, -1)
# sample_times already validated above

traj_V_G, traj_wsg_command = make_bot_trajectory(gripper_poses, finger_states, sample_times)



# Add system to the builder

# V_G_source defines a trajectory over gripper velocities. Add it to the system.
V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
# Add the DiffIK controller we just defined to the system
controller = builder.AddSystem(PseudoInverseController(plant))
# The HardwareStation expects robot commands in terms of joint angles.
# We define the `integrator` system to map from joint_velocities to joint_angles.
integrator = builder.AddSystem(Integrator(7))
# wsg_source defines a trajectory of finger positions. Add it to the system.
wsg_source = builder.AddSystem(TrajectorySource(traj_wsg_command))

# Connect everything
builder.Connect(V_G_source.get_output_port(), controller.GetInputPort("V_WG"))
builder.Connect(controller.GetOutputPort("iiwa.velocity"), integrator.get_input_port())
builder.Connect(integrator.get_output_port(), station.GetInputPort("iiwa.position"))
builder.Connect(station.GetOutputPort("iiwa.position_measured"), controller.GetInputPort("iiwa.position"))
builder.Connect(wsg_source.get_output_port(), station.GetInputPort("wsg.position"))


# Visualize everything
visualize_everything(station, plant)


diagram = builder.Build()


In [ ]:
# Define the simulator
simulator = Simulator(diagram)
context = simulator.get_mutable_context()
station_context = station.GetMyContextFromRoot(context)

# Get the plant context from the simulator's context
plant_context = plant.GetMyMutableContextFromRoot(context)
X_WGinitial = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("body"))



# Get the letter model and body
letter_model, letter_body = get_model_body(plant)


p_WL = p0_letter  # Use the same value from Block 1
plant.SetFreeBodyPose(
    plant_context,
    letter_body,
    RigidTransform(R_WL, p0_letter),
)


# Set integrator initial value
integrator.set_integral_value(
    integrator.GetMyContextFromRoot(context),
    plant.GetPositions(
        plant_context,
        plant.GetModelInstanceByName("iiwa"),
    ),
)

# Force publish to update visualization
diagram.ForcedPublish(context)
print(f"Simulation will run for {traj_V_G.end_time()} seconds")

# Run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(1.0)
# simulator.AdvanceTo(traj_V_G.end_time() if running_as_notebook else 0.01)

simulator.AdvanceTo(10)
meshcat.StopRecording()
meshcat.PublishRecording()